In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Deep Learning
from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
train = pd.read_csv('./train.csv', encoding='utf8')
test = pd.read_csv('./test.csv', encoding='utf8')
train_column_headers = list(train.columns.values)[1:]

# Analisis Exploratorio

In [ ]:
exploratory = train.copy()
threshold = 15

for column in train_column_headers:
  exploratory.loc[exploratory[column] < threshold, column] = 0
  exploratory.loc[exploratory[column] >= threshold, column] = 1

In [ ]:
# Cantidad de pixeles oscuros vs claros
train.head(1).transpose().value_counts()

In [ ]:
x_train = (exploratory.iloc[:,1:].values).astype('float32')
y_train = exploratory.iloc[:,0].values.astype('int32')

In [ ]:
# Convertir dataset a (numero de imagenes, pixeles ancho, pixeles largo) 
x_train = x_train.reshape(x_train.shape[0], 28, 28)

plt.subplot(331)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.title(y_train[1])

In [ ]:
# Se encuentran cuantos pixels blancos hay en cada imagen
exploratory = exploratory.astype('int32')

pixels = exploratory.sum(axis=1)
white_pixels = pd.DataFrame(data={'label': exploratory['label'], 'pixels': pixels})
white_pixels.groupby(by='label').describe()

In [ ]:
white_pixels['label'].value_counts()

In [ ]:
plt.rcParams["figure.figsize"] = (20, 20)
white_pixels.groupby(by='label').boxplot(column='pixels')

# Redes Neuronales

In [ ]:
x_train = (train.iloc[:,1:].values).astype('float32')
y_train = train.iloc[:,0].values.astype('int32')
x_test = test.values.astype('float32')

In [ ]:
#Convert train datset to (num_images, img_rows, img_cols) format 
x_train = x_train.reshape(x_train.shape[0], 28, 28)

for i in range(0, 9):
    plt.subplot(330 + (i+1))
    plt.imshow(x_train[i], cmap=plt.get_cmap('gray'))
    plt.title(y_train[i])

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28,1)
x_train.shape

In [ ]:
x_test = x_test.reshape(x_test.shape[0], 28, 28,1)
x_test.shape

In [ ]:
mean_px = x_train.mean().astype(np.float32)
std_px = x_train.std().astype(np.float32)

def standardize(x): 
  return (x-mean_px)/std_px

In [ ]:
y_train= to_categorical(y_train)
num_classes = y_train.shape[1]
num_classes